In [ ]:
import gradio as gr
import json
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import seaborn as sns
import sys

# Sets Current Directory to Parent Path
sys.path.insert(0, '..')

from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from src.utils.main import download_dataset, convert_dataset

In [ ]:
DATASET_URL = 'https://github.com/yaleemmlc/admissionprediction/raw/master/Results/5v_cleandf.RData'
FILE_PATH = '../data'
FILE_NAME = 'Results.rdata'

# Downloads the Original Dataset (RData File)
DATASET_RDATA_PATH = download_dataset(DATASET_URL, FILE_PATH, FILE_NAME)

# Converts the RData File to a New CSV File
DATASET_CSV_PATH = convert_dataset(DATASET_RDATA_PATH, DATASET_RDATA_PATH.replace('.rdata', '.csv'))

In [ ]:
df = pl.read_csv(DATASET_CSV_PATH)

df.head()

In [ ]:
df.describe()

In [ ]:
# Removes the First Column (Index)
df = df.drop("")

# columns = dict()
# for column in df.columns:
#     columns[column] = df[column].unique().sort().to_list()

# with open("columns.json", "w") as outfile:
#     outfile.write(json.dumps(columns, indent=4))

# Elimina las columnas donde todos sus elementos sean iguales
columns_to_remove = df.columns[df.n_unique() == 1]
df_witout_nunique = df.drop(columns=columns_to_remove)
df_witout_nunique.head()

In [ ]:
significantColumns = ['cc_chestpain', 'cc_breathingdifficulty', 'cc_syncope', 'cc_unresponsive', 'cc_seizure-newonset', 'cc_seizure-priorhxof', 'cc_seizures', 'cc_bleeding/bruising', 'cc_hyperglycemia', 'cc_hypertension', 'cc_hypotension', 'cc_strokealert', 'cc_overdose-accidental', 'cc_overdose-intentional', 'cc_suicidal']

In [ ]:
# If any of the significant columns are on true, automatically has the flag of emergency
df_witout_nunique = df_witout_nunique.to_pandas()
df_witout_nunique['emergency_flag_column'] = df_witout_nunique[significantColumns].any(axis=1)

print(df_witout_nunique['emergency_flag_column'].value_counts())

In [ ]:
df_converted = df_witout_nunique.copy()

# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()

# Identificar las variables categóricas
categorical_columns = []
for column in df_converted.columns:
    # Verificar si la columna es de tipo 'object' o 'category'
    if df_converted[column].dtype == 'object' or df_converted[column].dtype == 'category' :  
        categorical_columns.append(column)

# Convertir las variables categóricas usando LabelEncoder
for column in categorical_columns:
    df_converted[column] = label_encoder.fit_transform(df_converted[column])

print(df_converted['gender'].value_counts())

In [ ]:
columnsWithNan = df_converted.isna().any().pipe(lambda x: x.index[x])
print(len(columnsWithNan))

# Reemplazo los valores nulos con -999. Se podría intentar en una proxima iteracion con algo como col.mode()[0] o valores significativos reales
df_filled = df_converted.apply(lambda col: col.fillna(-999))

newColumnsWithNan = df_filled.isna().any().pipe(lambda x: x.index[x])
print(len(newColumnsWithNan))

In [ ]:
# División de datos
random_state = 42

X = df_filled.drop(columns=['emergency_flag_column'])
y = df_filled['emergency_flag_column']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [ ]:
# Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=100, random_state=random_state)
model.fit(X_train, y_train)

In [ ]:
# Validación del modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
FP = confusion[0, 1]
TN = confusion[0, 0]
FN = confusion[1, 0]

print(f"Verdaderos Positivos: {str(TP)}")

In [ ]:
# Codificación de variables categóricas
df_encoded = pd.get_dummies(df_witout_nunique)

In [ ]:
# Manejo de valores nulos
imputer = SimpleImputer(strategy='most_frequent')  
df_without_null = imputer.fit_transform(df_witout_nunique)
print(df_without_null)

In [ ]:
# Crear un DataFrame con las características y las etiquetas codificadas
df_plot = X.copy()
df_plot['emergency_flag'] = y

# Graficar la distribución de las clases
plt.figure(figsize=(8, 6))
sns.countplot(x='emergency_flag', data=df_plot)
plt.title('Distribución de Clases')
plt.xlabel('Clase')
plt.ylabel('Conteo')
plt.show()

In [ ]:
# Obtener las probabilidades de predicción en lugar de las etiquetas binarias
probabilities = model.predict_proba(X_test)[:, 1]

# Crear un DataFrame con las probabilidades, las etiquetas originales y otras columnas relevantes
result_df = pd.DataFrame({
    'Probability': probabilities,
    'Actual_Label': y_test,
    # También puedes agregar otras columnas relevantes del conjunto de prueba
})

threshold = 0.5
emergency_list = result_df[result_df['Probability'] > threshold].sort_values(by='Probability', ascending=False)

# Mostrar el listado de emergencias según su importancia
print(emergency_list)

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import sqlite3

class EmergencyClassifierApp:
    def __init__(self, model, database_path='patients.db'):
        self.model = model
        self.database_path = database_path

        # Create or connect to the SQLite database
        self.connection = sqlite3.connect(self.database_path)
        self.create_patient_table()

        self.root = tk.Tk()
        self.root.title("Emergency Classifier")

        self.name = None
        self.age = None

        self.questions = [
            "Does the patient have chest pain?",
            "Is the patient having difficulty breathing?",
            "Has the patient experienced fainting (syncope)?",
            "Is the patient unconscious or unresponsive?",
            "Has the patient had a recent seizure?",
            "Does the patient have a history of seizures?",
            "Has the patient had more than one seizure?",
            "Is the patient experiencing bleeding or bruising?",
            "Does the patient have elevated blood glucose levels?",
            "Does the patient have hypertension?",
            "Does the patient have hypotension?",
            "Is the patient a possible stroke alert?",
            "Has the patient had an accidental overdose?",
            "Has the patient had an intentional overdose?",
            "Does the patient have suicidal thoughts?"
        ]

        self.label = tk.Label(self.root, text="Enter Patient's Name:")
        self.label.pack(pady=10)

        self.name_entry = tk.Entry(self.root)
        self.name_entry.pack(pady=5)

        self.age_label = tk.Label(self.root, text="Enter Patient's Age:")
        self.age_label.pack(pady=5)

        self.age_entry = tk.Entry(self.root)
        self.age_entry.pack(pady=5)

        self.next_button = tk.Button(self.root, text="Next", command=self.ask_questions)
        self.next_button.pack(pady=10)

        self.result_label = tk.Label(self.root, text="")
        self.result_label.pack(pady=10)

        self.current_question_index = 0

    def create_patient_table(self):
        cursor = self.connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS patients (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                age INTEGER NOT NULL,
                is_emergency INTEGER NOT NULL
            )
        ''')
        self.connection.commit()

    def insert_patient_record(self, name, age, is_emergency):
        cursor = self.connection.cursor()
        cursor.execute('INSERT INTO patients (name, age, is_emergency) VALUES (?, ?, ?)', (name, age, is_emergency))
        self.connection.commit()

    def ask_questions(self):
        if not self.name or not self.age:
            self.name = self.name_entry.get()
            self.age = self.age_entry.get()

            if not self.name or not self.age:
                messagebox.showerror("Error", "Please enter both name and age.")
                return

            self.age = int(self.age)

        self.label.config(text=self.questions[self.current_question_index])
        self.current_question_index += 1

        if self.current_question_index == len(self.questions):
            self.next_button.config(command=self.show_result)
        else:
            self.next_button.config(command=self.answer_yes_no)

    def answer_yes_no(self):
        result = messagebox.askyesno("Question", self.questions[self.current_question_index])
        self.insert_patient_record(self.name, self.age, int(result))
        self.current_question_index += 1

        if self.current_question_index == len(self.questions):
            self.next_button.config(command=self.show_result)
            self.label.config(text="Click 'Next' to see the result.")
        else:
            self.label.config(text=self.questions[self.current_question_index])

    def show_result(self):
        if self.model:
            result = self.model.predict([0]*len(self.questions))  # Replace with actual prediction
            is_emergency = "Emergency" if result == 1 else "Is Not an Emergency"
        else:
            is_emergency = "Emergency (Model Not Defined)"

        position = self.get_patient_position()
        self.result_label.config(text=f"Result: {is_emergency}\nPosition in Database: {position}")

    def get_patient_position(self):
        cursor = self.connection.cursor()
        cursor.execute('SELECT id FROM patients ORDER BY id DESC LIMIT 1')
        result = cursor.fetchone()
        return result[0] if result else 1

    def __del__(self):
        # Close the database connection when the application is closed
        if self.connection:
            self.connection.close()

if __name__ == "__main__":
    app = EmergencyClassifierApp(model=None)
    app.root.mainloop()